### Iris 데이터 세트를 로딩하고 Pandas DataFrame으로 변환 후 scikit learn으로 학습 및 예측 수행.

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
df=spark.read.csv('C:\\Users\\jihun\\Downloads\\stats (7).csv',header=True,inferSchema=True)


from pyspark.sql.types import StructType,StructField, StringType
import re

for each in df.schema.names:
    df = df.withColumnRenamed(each,  re.sub(r'\s+([a-zA-Z_][a-zA-Z_0-9]*)\s*','',each.replace(' ', '')))
# 일부 컬럼에 있는 공백 없애기    
    
# pandas DataFrame을 spark DataFrame으로 부터 생성. 
pdf = df.select('*').toPandas()

In [3]:
df.cache()

DataFrame[last_name: string, first_name: string, player_id: int, year: int, p_k_percent: double, p_bb_percent: double, xwoba: double, z_swing_percent: double, z_swing_miss_percent: double, oz_swing_percent: double, oz_swing_miss_percent: double, oz_contact_percent: double, out_zone_percent: double, iz_contact_percent: double, in_zone_percent: double, edge_percent: double, whiff_percent: double, fastball_avg_speed: double, fastball_avg_spin: int, fastball_avg_break: double, breaking_avg_speed: double, breaking_avg_spin: int, breaking_avg_break: double, offspeed_avg_speed: double, offspeed_avg_spin: int, offspeed_avg_break: double, _c26: string]

In [4]:
from pyspark.sql.functions import count, isnan, isnull,when, col,max, avg, sum, min,lit,substring,concat,concat_ws

In [5]:
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

+---------+----------+---------+----+-----------+------------+-----+---------------+--------------------+----------------+---------------------+------------------+----------------+------------------+---------------+------------+-------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+----+
|last_name|first_name|player_id|year|p_k_percent|p_bb_percent|xwoba|z_swing_percent|z_swing_miss_percent|oz_swing_percent|oz_swing_miss_percent|oz_contact_percent|out_zone_percent|iz_contact_percent|in_zone_percent|edge_percent|whiff_percent|fastball_avg_speed|fastball_avg_spin|fastball_avg_break|breaking_avg_speed|breaking_avg_spin|breaking_avg_break|offspeed_avg_speed|offspeed_avg_spin|offspeed_avg_break|_c26|
+---------+----------+---------+----+-----------+------------+-----+---------------+--------------------+----------------+---------------------+------------------+-----

In [6]:
df=df.drop('_c26')

In [7]:
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).toPandas()

,last_name,first_name,player_id,year,p_k_percent,p_bb_percent,xwoba,z_swing_percent,z_swing_miss_percent,oz_swing_percent,...,whiff_percent,fastball_avg_speed,fastball_avg_spin,fastball_avg_break,breaking_avg_speed,breaking_avg_spin,breaking_avg_break,offspeed_avg_speed,offspeed_avg_spin,offspeed_avg_break
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,35,36,35,567,567,567


In [8]:
df=df.na.drop('any')

In [9]:
print((df.count(), len(df.columns)))

(3601, 26)


In [26]:
## SQL 사용이 가능할 수 있도록 sdf DataFrame을 DB View로 생성. 
df.createOrReplaceTempView('df_view')

In [41]:
breaking_null = spark.sql("""select * from df_view where 
player_id in (select player_id from df_view where breaking_avg_speed is null)
 order by player_id, year""")

In [39]:
breaking_null.toPandas()

,last_name,first_name,player_id,year,p_k_percent,p_bb_percent,xwoba,z_swing_percent,z_swing_miss_percent,oz_swing_percent,...,whiff_percent,fastball_avg_speed,fastball_avg_spin,fastball_avg_break,breaking_avg_speed,breaking_avg_spin,breaking_avg_break,offspeed_avg_speed,offspeed_avg_spin,offspeed_avg_break
0,Rodney,Fernando,407845,2016,26.1,13.1,0.289,61.1,23.1,29.6,...,30.0,95.1,2026,17.5,NaN,NaN,NaN,83.3,1818.0,16.6
1,Rodney,Fernando,407845,2017,28.1,11.3,0.278,60.2,23.3,27.3,...,31.0,94.5,2072,19.3,85.3,2269.0,4.8,83.5,1787.0,18.5
2,Rodney,Fernando,407845,2018,24.6,11.2,0.310,62.8,20.5,26.3,...,26.8,94.1,2067,18.8,86.5,2182.0,7.8,82.8,1735.0,17.3
3,Rodney,Fernando,407845,2019,23.3,13.3,0.316,63.7,17.9,31.0,...,26.3,93.8,2085,18.5,86.3,2254.0,7.9,83.0,1752.0,17.1
4,Madson,Ryan,425492,2016,18.1,7.4,0.290,67.9,19.0,34.0,...,25.1,94.5,2142,17.0,NaN,NaN,NaN,83.9,2028.0,16.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,Ferguson,Caleb,657571,2018,29.2,5.9,0.289,66.2,21.6,25.1,...,26.1,93.8,2289,19.1,75.6,2822.0,19.3,85.0,1763.0,16.7
71,Ferguson,Caleb,657571,2019,26.5,13.2,0.334,73.1,17.9,22.2,...,23.4,94.4,2311,19.6,76.2,3023.0,20.2,85.8,1917.0,19.2
72,Ferguson,Caleb,657571,2020,36.0,4.0,0.215,74.7,21.3,38.8,...,29.1,93.7,2352,16.6,NaN,NaN,NaN,91.8,2213.0,20.5
73,Ferguson,Caleb,657571,2022,26.1,12.0,0.273,68.7,20.7,23.0,...,26.9,94.8,2319,17.1,78.3,3084.0,19.7,NaN,NaN,NaN


In [119]:
breaking_null.where('breaking_avg_speed is not null').groupBy('player_id').agg(avg('breaking_avg_speed'),avg('breaking_avg_break'),avg('breaking_avg_spin')).collect()

[Row(player_id=656222, avg(breaking_avg_speed)=77.99999999999999, avg(breaking_avg_break)=13.200000000000001, avg(breaking_avg_spin)=2504.6666666666665),
 Row(player_id=435400, avg(breaking_avg_speed)=83.2, avg(breaking_avg_break)=6.2, avg(breaking_avg_spin)=1865.0),
 Row(player_id=657571, avg(breaking_avg_speed)=76.7, avg(breaking_avg_break)=19.733333333333334, avg(breaking_avg_spin)=2976.3333333333335),
 Row(player_id=641941, avg(breaking_avg_speed)=81.43333333333332, avg(breaking_avg_break)=11.700000000000001, avg(breaking_avg_spin)=2599.3333333333335),
 Row(player_id=517008, avg(breaking_avg_speed)=83.0, avg(breaking_avg_break)=13.3, avg(breaking_avg_spin)=2197.0),
 Row(player_id=622253, avg(breaking_avg_speed)=80.7, avg(breaking_avg_break)=10.4, avg(breaking_avg_spin)=2404.5),
 Row(player_id=607455, avg(breaking_avg_speed)=80.89999999999999, avg(breaking_avg_break)=11.5, avg(breaking_avg_spin)=2066.3333333333335),
 Row(player_id=593576, avg(breaking_avg_speed)=87.78, avg(breaking_

In [32]:
spark.sql("select * from df_view where breaking_avg_speed is not null")

DataFrame[last_name: string, first_name: string, player_id: int, year: int, p_k_percent: double, p_bb_percent: double, xwoba: double, z_swing_percent: double, z_swing_miss_percent: double, oz_swing_percent: double, oz_swing_miss_percent: double, oz_contact_percent: double, out_zone_percent: double, iz_contact_percent: double, in_zone_percent: double, edge_percent: double, whiff_percent: double, fastball_avg_speed: double, fastball_avg_spin: int, fastball_avg_break: double, breaking_avg_speed: double, breaking_avg_spin: int, breaking_avg_break: double, offspeed_avg_speed: double, offspeed_avg_spin: int, offspeed_avg_break: double]

### Spark DataFrame 생성 후 ML 알고리즘 학습 및 예측 수행. 
* iris_sdf DataFrame을 randomSplit()을 이용하여 train용과 test용 DataFrame으로 분할
* VectorAssembler를 이용하여 모든 feature 컬럼들을 하나의 feature vector로 변환
* Estimator 객체를 생성하고, fit() 메소드를 호출하여 ML Model 반환
* ML Model을 이용하여 테스트 DataFrame에 예측 수행.

In [ ]:
df=df.drop(*['last_name','first_name','player_id','year'])

In [150]:
train_sdf, test_sdf = df.randomSplit([0.8,0.2])
train_sdf.cache()
print(df.count(),train_sdf.count(),test_sdf.count())

3601 2918 683


In [151]:
from pyspark.ml.feature import VectorAssembler
df.columns

['p_k_percent',
 'p_bb_percent',
 'xwoba',
 'z_swing_percent',
 'z_swing_miss_percent',
 'oz_swing_percent',
 'oz_swing_miss_percent',
 'oz_contact_percent',
 'out_zone_percent',
 'iz_contact_percent',
 'in_zone_percent',
 'edge_percent',
 'whiff_percent',
 'fastball_avg_speed',
 'fastball_avg_spin',
 'fastball_avg_break',
 'breaking_avg_speed',
 'breaking_avg_spin',
 'breaking_avg_break',
 'offspeed_avg_speed',
 'offspeed_avg_spin',
 'offspeed_avg_break']

In [153]:
features = ['p_k_percent',
 'p_bb_percent',
 'z_swing_percent',
 'z_swing_miss_percent',
 'oz_swing_percent',
 'oz_swing_miss_percent',
 'oz_contact_percent',
 'out_zone_percent',
 'iz_contact_percent',
 'in_zone_percent',
 'edge_percent',
 'whiff_percent',
 'fastball_avg_speed',
 'fastball_avg_spin',
 'fastball_avg_break',
 'breaking_avg_speed',
 'breaking_avg_spin',
 'breaking_avg_break',
 'offspeed_avg_speed',
 'offspeed_avg_spin',
 'offspeed_avg_break']

In [154]:
from pyspark.ml.feature import VectorAssembler

vec_assembler = VectorAssembler(inputCols = features, outputCol = 'features')
train_feature_vector_df = vec_assembler.transform(train_sdf)
train_feature_vector_df.toPandas()

,p_k_percent,p_bb_percent,xwoba,z_swing_percent,z_swing_miss_percent,oz_swing_percent,oz_swing_miss_percent,oz_contact_percent,out_zone_percent,iz_contact_percent,...,fastball_avg_speed,fastball_avg_spin,fastball_avg_break,breaking_avg_speed,breaking_avg_spin,breaking_avg_break,offspeed_avg_speed,offspeed_avg_spin,offspeed_avg_break,features
0,4.0,8.9,0.450,75.0,7.8,24.2,24.0,76.0,52.4,92.2,...,89.7,2056,18.7,78.8,2202,7.3,84.1,1568,13.3,"[4.0, 8.9, 75.0, 7.8, 24.2, 24.0, 76.0, 52.4, ..."
1,4.8,12.9,0.356,62.2,10.1,23.5,14.8,85.2,47.5,87.3,...,94.8,2152,19.5,83.7,2212,11.5,82.9,1694,10.4,"[4.8, 12.9, 62.2, 10.1, 23.5, 14.8, 85.2, 47.5..."
2,5.2,9.1,0.373,71.4,9.4,25.2,18.5,81.5,47.3,90.6,...,89.0,1897,17.2,81.7,2116,10.7,82.0,1189,12.1,"[5.2, 9.1, 71.4, 9.4, 25.2, 18.5, 81.5, 47.3, ..."
3,5.6,11.2,0.363,64.7,4.4,20.5,27.8,72.2,55.9,94.4,...,87.0,2062,18.1,72.9,2732,19.0,78.9,1739,14.9,"[5.6, 11.2, 64.7, 4.4, 20.5, 27.8, 72.2, 55.9,..."
4,5.6,14.1,0.408,63.8,2.7,25.7,23.1,76.9,56.7,97.3,...,89.7,2130,16.2,79.8,2533,9.8,84.0,1395,14.1,"[5.6, 14.1, 63.8, 2.7, 25.7, 23.1, 76.9, 56.7,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913,45.5,10.7,0.229,73.1,34.3,31.2,67.9,32.1,52.9,65.7,...,96.4,2069,20.1,83.6,2494,5.0,87.7,1465,14.9,"[45.5, 10.7, 73.1, 34.3, 31.2, 67.9, 32.1, 52...."
2914,45.5,12.7,0.242,68.4,39.7,29.0,64.9,35.1,52.8,59.6,...,97.7,2354,19.6,89.6,2319,5.0,93.4,1669,19.0,"[45.5, 12.7, 68.4, 39.7, 29.0, 64.9, 35.1, 52...."
2915,46.7,9.8,0.231,73.0,33.5,29.8,54.2,45.3,48.4,66.3,...,94.5,2036,19.3,81.9,2498,7.9,85.2,1402,13.4,"[46.7, 9.8, 73.0, 33.5, 29.8, 54.2, 45.3, 48.4..."
2916,47.8,6.9,0.235,80.6,37.8,34.9,50.8,48.1,47.6,61.7,...,95.5,2118,19.6,81.9,2626,8.9,86.8,1471,15.4,"[47.8, 6.9, 80.6, 37.8, 34.9, 50.8, 48.1, 47.6..."


In [158]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features',labelCol='xwoba')
lr_model = lr.fit(train_feature_vector_df)
print(lr_model)

LinearRegressionModel: uid=LinearRegression_de5ec8e7d726, numFeatures=21


In [159]:
test_feature_vector_df = vec_assembler.transform(test_sdf)

In [162]:
predictions = lr_model.transform(test_feature_vector_df)
predictions.show(vertical=True)

-RECORD 0-------------------------------------
 p_k_percent           | 4.8                  
 p_bb_percent          | 22.6                 
 xwoba                 | 0.428                
 z_swing_percent       | 75.4                 
 z_swing_miss_percent  | 20.2                 
 oz_swing_percent      | 16.0                 
 oz_swing_miss_percent | 17.4                 
 oz_contact_percent    | 82.6                 
 out_zone_percent      | 55.0                 
 iz_contact_percent    | 79.8                 
 in_zone_percent       | 45.0                 
 edge_percent          | 34.0                 
 whiff_percent         | 19.6                 
 fastball_avg_speed    | 96.4                 
 fastball_avg_spin     | 2177                 
 fastball_avg_break    | 20.7                 
 breaking_avg_speed    | 88.0                 
 breaking_avg_spin     | 2296                 
 breaking_avg_break    | 6.9                  
 offspeed_avg_speed    | 90.8                 
 offspeed_avg

In [164]:
predictions.select('xwoba','prediction').show()

+-----+-------------------+
|xwoba|         prediction|
+-----+-------------------+
|0.428| 0.4465278983422323|
|0.502| 0.4227730013820157|
|0.356| 0.3835408996083842|
|0.465|0.39648710213420557|
|0.356|  0.396499960870603|
|0.321|0.37980909336788926|
|0.384| 0.3674308043290395|
|0.404| 0.3844618424420716|
|0.376|0.39294620192922736|
|0.349| 0.3789197582043897|
|0.395|0.38619246672742236|
|0.398| 0.3771178949371041|
|0.412|0.37346755584806246|
|0.366| 0.3724455331636456|
|0.461| 0.3880167847735869|
|0.379| 0.3745588497734005|
|0.345|0.37136747283332383|
|0.285| 0.3459396302976039|
|0.334|0.38075932328822404|
|0.368| 0.3664573988759159|
+-----+-------------------+
only showing top 20 rows



In [170]:
from pyspark.ml.evaluation import RegressionEvaluator
# RMSE
evaluator = RegressionEvaluator(predictionCol='prediction',
                               labelCol='xwoba',
                               metricName='rmse')
# MAE                            
evaluator = RegressionEvaluator(predictionCol='prediction',
                               labelCol='xwoba',
                               metricName='mae')
# R2 Score
evaluator = RegressionEvaluator(predictionCol='prediction',
                               labelCol='xwoba',
                               metricName='r2')

print(f"RMSE: {evaluator.evaluate(predictions)}")
print(f"MAE: {evaluator.evaluate(predictions)}")  
print(f"R2 score: {evaluator.evaluate(predictions)}")

RMSE: 0.5360404390079032
MAE: 0.5360404390079032
R2 score: 0.5360404390079032


In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

train_sdf, test_sdf = df.randomSplit([0.8,0.2])
train_sdf.cache()
print(df.count(),train_sdf.count(),test_sdf.count())

3601 2878 723


In [40]:
train_sdf

DataFrame[last_name: string, first_name: string, player_id: int, year: int, p_k_percent: double, p_bb_percent: double, xwoba: double, z_swing_percent: double, z_swing_miss_percent: double, oz_swing_percent: double, oz_swing_miss_percent: double, oz_contact_percent: double, out_zone_percent: double, iz_contact_percent: double, in_zone_percent: double, edge_percent: double, whiff_percent: double, fastball_avg_speed: double, fastball_avg_spin: int, fastball_avg_break: double, breaking_avg_speed: double, breaking_avg_spin: int, breaking_avg_break: double, offspeed_avg_speed: double, offspeed_avg_spin: int, offspeed_avg_break: double]

In [60]:
feature_cols=['p_k_percent',
 'p_bb_percent',
 'z_swing_percent',
 'z_swing_miss_percent',
 'oz_swing_percent',
 'oz_swing_miss_percent',
 'oz_contact_percent',
 'out_zone_percent',
 'iz_contact_percent',
 'in_zone_percent',
 'edge_percent',
 'whiff_percent',
 'fastball_avg_speed',
 'fastball_avg_spin',
 'fastball_avg_break',
 'breaking_avg_speed',
 'breaking_avg_spin',
 'breaking_avg_break',
 'offspeed_avg_speed',
 'offspeed_avg_spin',
 'offspeed_avg_break']

In [63]:
stage_1 = VectorAssembler(inputCols=feature_cols,outputCol='features')
stage_2 = LinearRegression(featuresCol = 'features', labelCol = 'xwoba')

pipeline = Pipeline(stages=[stage_1,stage_2])
pipeline_model = pipeline.fit(train_sdf)
predictions = pipeline_model.transform(test_sdf)

print(type(pipeline))
print(type(pipeline_model))

<class 'pyspark.ml.pipeline.Pipeline'>
<class 'pyspark.ml.pipeline.PipelineModel'>


In [65]:
predictions.select('last_name','first_name','xwoba','prediction').show()

+---------+----------+-----+-------------------+
|last_name|first_name|xwoba|         prediction|
+---------+----------+-----+-------------------+
|     Adam|     Jason|0.351| 0.3223198641661611|
|  Agrazal|     Dario|0.346| 0.3489259829766347|
|     Akin|    Keegan|0.349|0.33924643446279745|
|   Albers|    Andrew| 0.39| 0.3315872269465867|
|   Alcala|     Jorge|0.288|0.29364562851986875|
|   Alcala|     Jorge|0.293| 0.2773451282525734|
|Alcantara|     Sandy| 0.31| 0.3143534345735155|
|Alexander|     Scott|0.289|0.32050070014616594|
|Alexander|     Scott|0.239|0.28901089752825904|
|Alexander|     Tyler|0.355|0.34469194577866524|
|   Allard|     Kolby|0.329|  0.343844538814416|
|   Allard|     Kolby|0.332|0.33939499300781506|
|  Almonte|     Yency|0.292| 0.3170894169205649|
|  Almonte|     Yency|0.351| 0.3394383275005026|
|  Alvarez|      Jose|0.309|0.31564667643159616|
|  Alvarez|      Jose|0.296|0.31135882595309283|
|  Alvarez|      Jose|  0.4|  0.332412400964186|
|  Alzolay|    Adber

In [66]:
from pyspark.ml.evaluation import RegressionEvaluator
# RMSE
evaluator = RegressionEvaluator(predictionCol='prediction',
                               labelCol='xwoba',
                               metricName='rmse')
# MAE                            
evaluator = RegressionEvaluator(predictionCol='prediction',
                               labelCol='xwoba',
                               metricName='mae')
# R2 Score
evaluator = RegressionEvaluator(predictionCol='prediction',
                               labelCol='xwoba',
                               metricName='r2')

print(f"RMSE: {evaluator.evaluate(predictions)}")
print(f"MAE: {evaluator.evaluate(predictions)}")  
print(f"R2 score: {evaluator.evaluate(predictions)}")

RMSE: 0.46915268531089926
MAE: 0.46915268531089926
R2 score: 0.46915268531089926


In [67]:
pipeline_model.stages

[VectorAssembler_2f4ccadf81a0,
 LinearRegressionModel: uid=LinearRegression_26140d26bd7b, numFeatures=21]